On import les bibliotheques :

In [ ]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import json

from pandas import json_normalize
import pandas as pd    
import json

import requests
import shutil
import os

import urllib.request

On crée la fonction de création du JSON

In [ ]:
####################################### CRÉATION JSON #################################################

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def crea_json():
    endpoint_url = "https://query.wikidata.org/sparql"

    query = """SELECT ?gateau ?gateauLabel ?gateauDescription ?image ?origineLabel
    {
    ?gateau wdt:P279/wdt:P31* wd:Q477248; #des pâtisseries
            wdt:P18 ?image; #l'image
            wdt:P495 ?origine. # l'origine du gateau
                
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en"}

    }
    LIMIT 1000"""

    results = get_results(endpoint_url, query)
    dico_json ={}

    for result in results["results"]["bindings"]:
        #print(result)
        # on récupère le nom du gateau
        gateau = result['gateauLabel']['value']
        # les valeurs qui nous interessent dans notre requête
        dico={}
        for key, val in result.items():
            dico[key]=val['value']
        dico_json[gateau]=dico
        #print("\n", gateau,'\n')
        #print(dico_json[gateau], '\n')

    #print(' dico json \n')
    #print(dico_json)
    with open('./fichier.json', 'w') as fichier :
        json.dump(dico_json, fichier)

crea_json()

On télécharge nos images

In [ ]:
def get_imgs():
    # the name of the json with all the data
    json_path="fichier.json"
    # we get the json
    json_data = json.load(open(json_path))


    for key, info in json_data.items():
        #print('\n', key,'\n', info, '\n')
        name = info['gateauLabel']# the name to save the image
        print(info['image'],"\n")
        urllib.request.urlretrieve(info['image'], "./script_python/images/"+name+".jpg")

get_imgs()

On cherche à récupérer les méta datas de chaque images

In [2]:
from PIL import Image
from PIL.ExifTags import TAGS 
import pandas as pd
import numpy
#from exif import Image

import os



def recup_matadata_img(path):
    """ Cette fonction récpère les métadatas d'une image si elle en a s et les stockes dans un data frame et le retourne
    INPUT :
        path : the path of the image to get the meta-data from
    OUTPUT :
        df : the f=data frame with all the meta-data we could get
    """
    #path =""# the image's path

    image = Image.open(path)# reading the image
   
    exifdata = image.getexif() #extracting the data
    # two list for the creation of the data frame
    keys=[]
    values=[]
    #print(exifdata)

    for tag_id in exifdata: # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag
        
        if isinstance(data, bytes): # decoding the tags data
            data = data.decode()
        keys.append(tag)
        values.append(str(data))
    
    ar=numpy.array([values])# the array for the data frame
    df = pd.DataFrame(ar, columns=keys)# the data frame
        
    return df

#################################### Some tests ###########################
print('result \n')
#recup_matadata_img("images/Alivenci.jpg")
print(recup_matadata_img("images/Alivenci.jpg"))
###########################################################################

def concat_lst_frame(lst):
    """ Cette fonction stocke la liste sous la forme d'un tuple"""
    print('Concaténation des frames \n')
    tup = ()
    for df in lst:
        tup =(tup, df)
    #print(tup)
    return tup



                     PrintImageMatching ResolutionUnit ExifOffset  \
0  PrintIM 0250                            2        394   

                                    ImageDescription               Make  \
0                                                ...  Minolta Co., Ltd.   

         Model     Software Orientation             DateTime YCbCrPositioning  \
0  DiMAGE F300  GIMP 2.10.8           1  2019:01:05 21:59:51                2   

  XResolution YResolution  
0        72.0        72.0  


On cherche à sauvegarder ces datas

In [3]:
import csv

def recup_matadata_img(path):
    """ Cette fonction récpère les métadatas d'une image si elle en a s et les stockes dans un data frame et le retourne
    INPUT :
        path : the path of the image to get the meta-data from
    OUTPUT :
        df : the f=data frame with all the meta-data we could get
    """
    #path =""# the image's path

    image = Image.open(path)# reading the image
   
    exifdata = image.getexif() #extracting the data
    # two list for the creation of the data frame
    keys=[]
    values=[]
    #print(exifdata)

    for tag_id in exifdata: # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag
        
        if isinstance(data, bytes): # decoding the tags data
            data = data.decode()
        keys.append(tag)
        values.append(str(data))
    
    ar=numpy.array([values])# the array for the data frame
    df = pd.DataFrame(ar, columns=keys)# the data frame
        
    return df

def save_meta_data(path):

    with open('meta_data.csv', 'w') as file:
        writer_obj=csv.writer(file, delimiter='|')

        dico = recup_matadata_img(path)#getting the meta_datas of the image
        header, values = [], []

        for key, value in dico.items():# the values to write into our csv file
                header.append(key)
                values.append(value)
                
            
        #print(values)
        writer_obj.writerow(header)# we save it in the csv file
        writer_obj.writerow(values)

############################################ Some tests #####################################
save_meta_data("images/Alivenci.jpg",)
#############################################################################################

def stock_meta_data(path):
    
    files = os.listdir(path)
    for name in files :
        try :
            print(name)
            recup_matadata_img("images/"+name)
            print("done")
        except :
            print(name)
            print("error")

Error: need to escape, but no escapechar set

We try with data frames

In [11]:
import os
import pandas as pd


def recup_matadata_img(path):
    """ Cette fonction récpère les métadatas d'une image si elle en a s et les stockes dans un data frame et le retourne
    INPUT :
        path : the path of the image to get the meta-data from
    OUTPUT :
        df : the f=data frame with all the meta-data we could get
    """
    #path =""# the image's path

    image = Image.open(path)# reading the image
   
    exifdata = image.getexif() #extracting the data
    # two list for the creation of the data frame
    keys=[]
    values=[]
    #print(exifdata)

    for tag_id in exifdata: # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag
        
        if isinstance(data, bytes): # decoding the tags data
            data = data.decode()
        keys.append(tag)
        values.append(str(data))
    
    ar=numpy.array([values])# the array for the data frame
    df = pd.DataFrame(ar, columns=keys)# the data frame
        
    return df



def stock_meta_datas(path, csv_name):
    """ This function stock the meta-datas of the images of a whole folder
    INPUT:
        path : the path of the folder we get the images from
        csv_name : the name of the csv file whre we will stock the image's meta-data
    OUTPUT:
        NONE, the meta-datas are stored into a csv file
    """
    files = os.listdir(path)# we get all the photos from the folder
    lst = []
    counter=0# to check we do not an loose an existing frame
    for name in files :
        try :
            #print(name)
            lst.append(recup_matadata_img("images/"+name))
            counter+=1
            #print("done")
        except :
            #print(name) 
            #print("error")
            pass
    frame=pd.concat(lst) # creating a big frame with all the images data-frame
    #print(frame)
    #print("On a {} frame au final ".format(len(lst)) )
    #print("\n normaleent on doit avoir {} frames".format(counter))
    frame.to_csv(csv_name, sep='|',encoding='utf-8', index=False, quoting=csv.QUOTE_NONE, escapechar=" ")#writting into our csv file

#####################################################Some test ################################
stock_meta_datas('images/', "meta_data.csv")

                      PrintImageMatching ResolutionUnit ExifOffset  \
0                                    NaN            NaN        NaN   
0   PrintIM 0250                            2        394   
0                                    NaN              2        324   
0                                    NaN              2        212   
0                                    NaN            NaN        NaN   
..                                   ...            ...        ...   
0                                    NaN            NaN        NaN   
0                                    NaN              2        210   
0               PrintIM 0250                     2        294   
0                                    NaN              2        240   
0                                    NaN              2        360   

                                     ImageDescription                Make  \
0                                                 NaN                 NaN   
0    